Inputs
- Control file: A CSV file that includes an 'Accession' column with a list of image accessions for radiomic feature extraction 
- Images folder - A folder containing subdirectories with images and segmentations in the format `<images_folder>/<accession>/<file.nii.gz>` where `file` is either `t1ce_img`, `flair_img`, `core_seg`, and/or `whole_seg`. 
- Path for the output CSV

Output: 
- CSV file with radiomic features extracted after masking each image with the associated segmentation. The file indicates the associated accession and VOI used 

In [ ]:
import nibabel as nib 
from radiomics import featureextractor
import pandas as pd
import os
from tqdm import tqdm

In [ ]:
control_file_path = 'lung_bm.csv'
images_folder_path = '/Users/saahilchadha/Desktop/visageAPI/docker_automatic/docker_mount/images/'

radiomics_csv_path = '/Users/saahilchadha/Desktop/visageAPI/docker_automatic/docker_mount/radiomics.csv'

In [ ]:
accessions_df = pd.read_csv(control_file_path)
accessions_df

In [ ]:
def extract_features(img_path, seg_path, prefix):
    img = nib.load(img_path)
    seg = nib.load(seg_path)

    image_data = img.get_fdata()
    segmentation_data = seg.get_fdata()

    # Mask the image using the segmentation
    masked_image = image_data * (segmentation_data > 0)

    # Set up the PyRadiomics feature extractor
    extractor = featureextractor.RadiomicsFeatureExtractor()
    extractor.enableAllFeatures()  # Enable all feature classes

    # Extract features
    result = extractor.execute(img_path, seg_path)

    filtered_features = {f"{prefix}_{featureName}": featureValue for featureName, featureValue in result.items() if not featureName.startswith("diagnostics")}
    return filtered_features
    

In [ ]:
features_df_lst = []
for accession in tqdm(accessions_df['Accession']):
    features_entry = {'accession': accession}
    
    expected_t1ce_img_path = os.path.join(images_folder_path, accession, 't1ce_img.nii.gz')
    if os.path.exists(expected_t1ce_img_path):
        img_path = expected_t1ce_img_path
        seg_path = os.path.join(images_folder_path, accession, 'core_seg.nii.gz')

        filtered_features = extract_features(img_path, seg_path, 't1ce')

        features_entry.update(filtered_features)
        features_entry
    
    expected_flair_img_path = os.path.join(images_folder_path, accession, 'flair_img.nii.gz')
    if os.path.exists(expected_flair_img_path):
        img_path = expected_flair_img_path
        seg_path = os.path.join(images_folder_path, accession, 'whole_seg.nii.gz')

        filtered_features = extract_features(img_path, seg_path, 'flair')

        features_entry.update(filtered_features)
        features_entry
    features_df_lst.append(features_entry)
pd.DataFrame(features_df_lst).to_csv(radiomics_csv_path)
